In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
def preprocess(image, size = (64,64)):
    labels = 0
    label = tf.constant((labels), dtype = tf.int32)
    image_cropped = tf.image.crop_to_bounding_box(image, 20, 0, 178, 178)
    image_resized = tf.image.resize(image_cropped, size=size)
    #image_resized =  image_resized[None, :, :]
    image_resized = tf.expand_dims(image_resized, axis=0)
    label1 = tf.expand_dims(label, axis=0)
    #label = label[None,:]
    return (image_resized/255,label1)


model = keras.models.load_model('model.h5')

import cv2
import numpy as np
import imutils

# Load the cascade


# To capture video from webcam. 
cap = cv2.VideoCapture(0)


CAPTURE_WIDTH = 900
ROI_LONG = 178 # Region Of Interest
MARGIN = 350
TOP = 100
RIGHT = CAPTURE_WIDTH - MARGIN
BOTTOM = TOP + 218
LEFT = RIGHT - ROI_LONG

CAMERA = cv2.VideoCapture(0)

while(True):
    _, frame = CAMERA.read()
    frame = imutils.resize(frame, CAPTURE_WIDTH)
    (height, width) = frame.shape[:2]
    cv2.rectangle(frame, (LEFT, TOP), (RIGHT, BOTTOM), (0,255,0), 2)
    
    img1 = frame[TOP:BOTTOM,LEFT:RIGHT]
    img2 = preprocess(frame[TOP:BOTTOM,LEFT:RIGHT])
    #print(img2)
    #d = d.map(lambda x,y:tf.stack([x,y],axis=-1))
    pred_logits = model.predict(img2[0])
    probas = tf.sigmoid(pred_logits)
    probas = probas.numpy().flatten()*100
    
    if probas >= 50:
        pred = 'Male'
    else:
        pred = 'female'
    LABEL_TEXT = str(pred)
    LABEL_COLOR = (0,255,0)
    cv2.putText(frame, LABEL_TEXT, (LEFT, TOP-7), cv2.FONT_HERSHEY_SIMPLEX, 1, LABEL_COLOR, 2)
    cv2.imshow('frame', frame)
    cv2.imshow('frame1', img1)
    # if the user pressed "q", then stop looping
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def preprocess(example, size = (64,64), mode = 'train'):
    image = example['image']
    #print(image.shape)
    label = example['attributes']['Male']
    #print(label)
    if mode == 'train':
        image_crop = tf.image.random_crop(image, size = (178, 178, 3))
        image_resized = tf.image.resize(image_crop, size = size)
        image_flip = tf.image.random_flip_left_right(image_resized)
        return (image_flip/255, tf.cast(label, tf.int32))
    else:
        image_cropped = tf.image.crop_to_bounding_box(
            image, 0, 0, 178, 178)
        image_resized = tf.image.resize(
            image_cropped, size=size)
        print(image_resized/255.0, tf.cast(label, tf.int32))
        return (image_resized/255.0, tf.cast(label, tf.int32))
    
import tensorflow_datasets as tfds
dataset = tfds.builder('celeb_a')
dataset.download_and_prepare()
data = dataset.as_dataset(shuffle_files=False)
data_test = data['test']
ds_test = data_test.map(
    lambda x:preprocess(x, mode='eval')).batch(32)
results = model.evaluate(ds_test, verbose=0)
print('Test Acc: {:.2f}%'.format(results[1]*100))